In [76]:
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import time
from numba import jit
import xgbfir
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
import os

In [42]:
# importing the files
start = time.time()
train_data = pd.read_csv("/home/saurabhg/PuertoSergo/train.csv")
test_data = pd.read_csv("/home/saurabhg/PuertoSergo/test.csv")
print(time.time()-start)

9.19328904152


In [69]:
target = train_data["target"]
#del trn_df["target"]

In [54]:
train_copy = train_data
train_copy = train_copy.replace(-1, np.NaN)
train_copy.isnull().sum()/train_copy.shape[0]*100
# train_copy.isnull().sum()

id                 0.000000
target             0.000000
ps_ind_01          0.000000
ps_ind_02_cat      0.036290
ps_ind_03          0.000000
ps_ind_04_cat      0.013945
ps_ind_05_cat      0.975955
ps_ind_06_bin      0.000000
ps_ind_07_bin      0.000000
ps_ind_08_bin      0.000000
ps_ind_09_bin      0.000000
ps_ind_10_bin      0.000000
ps_ind_11_bin      0.000000
ps_ind_12_bin      0.000000
ps_ind_13_bin      0.000000
ps_ind_14          0.000000
ps_ind_15          0.000000
ps_ind_16_bin      0.000000
ps_ind_17_bin      0.000000
ps_ind_18_bin      0.000000
ps_reg_01          0.000000
ps_reg_02          0.000000
ps_reg_03         18.106490
ps_car_01_cat      0.017977
ps_car_02_cat      0.000840
ps_car_03_cat     69.089837
ps_car_04_cat      0.000000
ps_car_05_cat     44.782531
ps_car_06_cat      0.000000
ps_car_07_cat      1.930237
ps_car_08_cat      0.000000
ps_car_09_cat      0.095596
ps_car_10_cat      0.000000
ps_car_11_cat      0.000000
ps_car_11          0.000840
ps_car_12          0

In [51]:
train_copy.shape[0]

595212

In [47]:
print(os.getcwd()+'/')

/home/puneetj/PortoSegeo/


In [100]:
# Defining the Normalized Gini method
@jit
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)

def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_lgb(act, preds):
#    labels = dtrain.get_label()
    gini_score = gini_normalized(act, preds)
    return 'gini', gini_score,True


In [ ]:
# Defining the Normalized Gini method
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)

def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return [('gini', gini_score)]
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))
 


In [96]:
a = [.7,.6,.77,.8]
b = [1,1,0,0]
eval_gini(b,a)

-1

In [52]:
target_train = train_data['target'].values
id_test = test_data['id'].values

df_train=train_data.drop(['target','id'],axis=1)
df_test=test_data.drop(['id'], axis = 1)
combine= pd.concat([df_train,df_test],axis=0)

In [53]:
# combine.shape
# df_train.shape
# df_test.shape
combine.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,2,2,5,1,0,0,1,0,0,0,...,9,1,5,8,0,1,1,0,0,1
1,1,1,7,0,0,0,0,1,0,0,...,3,1,1,9,0,1,1,0,1,0
2,5,4,9,1,0,0,0,1,0,0,...,4,2,7,7,0,1,1,0,1,0
3,0,1,2,0,0,1,0,0,0,0,...,2,2,4,9,0,0,0,0,0,0
4,0,2,0,1,0,1,0,0,0,0,...,3,1,1,3,0,0,0,1,1,0


In [58]:
# Feature engg code --- will be standard for all the models

combine['ps_ind_06_13_16_18_bin_sum'] = combine.ps_ind_06_bin + combine.ps_ind_07_bin + combine.ps_ind_08_bin + combine.ps_ind_09_bin + combine.ps_ind_10_bin + combine.ps_ind_11_bin + combine.ps_ind_12_bin + combine.ps_ind_13_bin + combine.ps_ind_16_bin + combine.ps_ind_17_bin + combine.ps_ind_18_bin   

combine['ps_reg_01_square'] = combine['ps_reg_01']*combine['ps_reg_01']
combine['ps_reg_01_cube'] = combine['ps_reg_01']*combine['ps_reg_01']*combine['ps_reg_01']

combine['ps_reg_02_square'] = combine['ps_reg_02']*combine['ps_reg_02']
combine['ps_reg_02_cube'] = combine['ps_reg_02']*combine['ps_reg_02']*combine['ps_reg_02']

combine['ps_reg_03_square'] = combine['ps_reg_03']*combine['ps_reg_03']
combine['ps_reg_03_cube'] = combine['ps_reg_03']*combine['ps_reg_03']*combine['ps_reg_03']

combine['ps_car_12_square'] = combine['ps_car_12']*combine['ps_car_12']
combine['ps_car_12_cube'] = combine['ps_car_12']*combine['ps_car_12']*combine['ps_car_12']

combine['ps_car_13_square'] = combine['ps_car_13']*combine['ps_car_13']
combine['ps_car_13_cube'] = combine['ps_car_13']*combine['ps_car_13']*combine['ps_car_13']

combine['ps_car_14_square'] = combine['ps_car_14']*combine['ps_car_14']
combine['ps_car_14_cube'] = combine['ps_car_14']*combine['ps_car_14']*combine['ps_car_14']

combine['ps_car_15_square'] = combine['ps_car_15']*combine['ps_car_15']
combine['ps_car_15_cube'] = combine['ps_car_15']*combine['ps_car_15']*combine['ps_car_15']

combine['ps_calc_15_20_bin_sum'] = combine.ps_calc_15_bin + combine.ps_calc_16_bin + combine.ps_calc_17_bin + combine.ps_calc_18_bin + combine.ps_calc_19_bin + combine.ps_calc_20_bin

combine['ps_car_11_ct_num'] = combine.ps_car_11_cat
combine['ps_car_11_ct_num_square'] = combine['ps_car_11_ct_num']*combine['ps_car_11_ct_num']

combine = combine.drop(combine[[x for x in combine.columns if "_calc_" in x]] ,axis =1)

In [64]:
combs = [
    ('ps_reg_01', 'ps_car_02_cat'),  
    ('ps_reg_01', 'ps_car_04_cat'),
]
start = time.time()
for n_c, (f1, f2) in enumerate(combs):
    name1 = f1 + "_plus_" + f2
    print('current feature %60s %4d in %5.1f'% (name1, n_c + 1, (time.time() - start) / 60))
    print('\r' * 75)
    combine[name1] = combine[f1].apply(lambda x: str(x)) + "_" + combine[f2].apply(lambda x: str(x))
#    sub_df[name1] = sub_df[f1].apply(lambda x: str(x)) + "_" + sub_df[f2].apply(lambda x: str(x))
    # Label Encode
    lbl = LabelEncoder()
    lbl.fit(list(combine[name1].values))
    combine[name1] = lbl.transform(list(combine[name1].values))
#   sub_df[name1] = lbl.transform(list(sub_df[name1].values))

current feature                                 ps_reg_01_plus_ps_car_02_cat    1 in   0.0

current feature                                 ps_reg_01_plus_ps_car_04_cat    2 in   0.1



In [67]:
# Creating the final data sets for training the testing
trn_df=combine[:df_train.shape[0]]
sub_df=combine[df_train.shape[0]:]

In [72]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior
    """
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [73]:
f_cats = [f for f in combine.columns if "_cat" in f]

for f in f_cats:
    trn_df[f + "_avg"], sub_df[f + "_avg"] = target_encode(trn_series=trn_df[f],
                                             tst_series=sub_df[f],
                                             target=target,
                                             min_samples_leaf=200,
                                             smoothing=10,
                                             noise_level=0)

/home/puneetj/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [74]:
trn_df.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_car_04_cat_avg,ps_car_05_cat_avg,ps_car_06_cat_avg,ps_car_07_cat_avg,ps_car_08_cat_avg,ps_car_09_cat_avg,ps_car_10_cat_avg,ps_car_11_cat_avg,ps_reg_01_plus_ps_car_02_cat_avg,ps_reg_01_plus_ps_car_04_cat_avg
0,2,2,5,1,0,0,1,0,0,0,...,0.033451,0.040454,0.033851,0.034766,0.044933,0.033344,0.036464,0.038642,0.035967,0.035641
1,1,1,7,0,0,0,0,1,0,0,...,0.033451,0.031709,0.031720,0.034766,0.034735,0.036319,0.036464,0.023936,0.036362,0.036098
2,5,4,9,1,0,0,0,1,0,0,...,0.033451,0.031709,0.034294,0.034766,0.034735,0.036319,0.036464,0.031281,0.037539,0.035427
3,0,1,2,0,0,1,0,0,0,0,...,0.033451,0.040454,0.031720,0.034766,0.034735,0.035104,0.036464,0.045015,0.038214,0.038347
4,0,2,0,1,0,1,0,0,0,0,...,0.033451,0.031709,0.034294,0.034766,0.034735,0.036319,0.036464,0.026266,0.035967,0.035641


In [147]:
n_splits = 5
n_estimators = 1000
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=15) 
imp_df = np.zeros((len(trn_df.columns), n_splits))
lgb_evals = np.zeros((n_estimators, n_splits))
oof = np.empty(len(trn_df))
sub_preds = np.zeros(len(sub_df))
increase = True
np.random.seed(0)

for fold_, (trn_idx, val_idx) in enumerate(folds.split(target, target)):
    trn_dat, trn_tgt = trn_df.iloc[trn_idx], target.iloc[trn_idx]
    val_dat, val_tgt = trn_df.iloc[val_idx], target.iloc[val_idx]

    clf = lgb.LGBMModel(n_estimators=n_estimators,
                        num_leaves=20,
                        objective="binary",
                        learning_rate=.01, 
                        subsample=.8, 
                        colsample_bytree=.8,
                        min_child_samples= 500,
                        is_training_metric = True,
                     #   gamma=1,
                        reg_alpha=0,
                        reg_lambda=1,
                      #  nthread=2
                       )
    # Upsample during cross validation to avoid having the same samples
    # in both train and validation sets
    # Validation set is not up-sampled to monitor overfitting
    if increase:
        # Get positive examples
        pos = pd.Series(trn_tgt == 1)
        # Add positive examples
        trn_dat = pd.concat([trn_dat, trn_dat.loc[pos]], axis=0)
        trn_tgt = pd.concat([trn_tgt, trn_tgt.loc[pos]], axis=0)
        # Shuffle data
        idx = np.arange(len(trn_dat))
        np.random.shuffle(idx)
        trn_dat = trn_dat.iloc[idx]
        trn_tgt = trn_tgt.iloc[idx]
        
    clf.fit(trn_dat, trn_tgt, 
            eval_set=[(trn_dat, trn_tgt), (val_dat, val_tgt)],
            categorical_feature = f_cats,
            eval_metric=gini_lgb,
            early_stopping_rounds=None,
            verbose=False)
    # Find best round for validation setA
    lgb_evals[:, fold_] = clf.evals_result_["valid_1"]["gini"]
    #print(clf.evals_result_ )
    #Xgboost provides best round starting from 0 so it has to be incremented
    best_round = np.argsort(lgb_evals[:, fold_])[::-1][0]

    # Predict OOF and submission probas with the best round
    oof[val_idx] = clf.predict(val_dat, num_iteration=best_round)
    # Update submission
    sub_preds += clf.predict(sub_df, num_iteration=best_round)/ n_splits

    # Display results
    print("Fold %2d : %.6f @%4d / best score is %.6f @%4d"
          % (fold_ + 1,
             gini_normalized(val_tgt, oof[val_idx]),
             n_estimators,
             lgb_evals[best_round, fold_],
             best_round))
          
print("Full OOF score : %.6f" % gini_normalized(target, oof))

Fold  1 : 0.277924 @1000 / best score is 0.277946 @ 999
Fold  2 : 0.284078 @1000 / best score is 0.284104 @ 922
Fold  3 : 0.308356 @1000 / best score is 0.308357 @ 912
Fold  4 : 0.283308 @1000 / best score is 0.283322 @ 995
Fold  5 : 0.280104 @1000 / best score is 0.280122 @ 999
Full OOF score : 0.286584


In [148]:
mean_eval = np.mean(lgb_evals, axis=1)
std_eval = np.std(lgb_evals, axis=1)
best_round = np.argsort(lgb_evals)[::-1][0]
#print("Best mean score : %.6f + %.6f @%4d"% (mean_eval[best_round], std_eval[best_round], best_round))

TypeError: float argument required, not numpy.ndarray

In [152]:
importances = sorted([(trn_df.columns[i], imp) for i, imp in enumerate(imp_df.mean(axis=1))],
                     key=lambda x: x[1])

for f, imp in importances[::-1]:
    print("%-34s : %10.4f" % (f, imp))
    
sub_df["target"] = sub_preds

sub_df[["target"]].to_csv("submission_10262017_lgb_5s_first.csv", index=False, float_format="%.9f")

ps_reg_01_plus_ps_car_04_cat_avg   :     0.0000
ps_reg_01_plus_ps_car_02_cat_avg   :     0.0000
ps_car_11_cat_avg                  :     0.0000
ps_car_10_cat_avg                  :     0.0000
ps_car_09_cat_avg                  :     0.0000
ps_car_08_cat_avg                  :     0.0000
ps_car_07_cat_avg                  :     0.0000
ps_car_06_cat_avg                  :     0.0000
ps_car_05_cat_avg                  :     0.0000
ps_car_04_cat_avg                  :     0.0000
ps_car_03_cat_avg                  :     0.0000
ps_car_02_cat_avg                  :     0.0000
ps_car_01_cat_avg                  :     0.0000
ps_ind_05_cat_avg                  :     0.0000
ps_ind_04_cat_avg                  :     0.0000
ps_ind_02_cat_avg                  :     0.0000
ps_reg_01_plus_ps_car_04_cat       :     0.0000
ps_reg_01_plus_ps_car_02_cat       :     0.0000
ps_car_11_ct_num_square            :     0.0000
ps_car_11_ct_num                   :     0.0000
ps_car_15_cube                     :    

/home/puneetj/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [154]:
sub = pd.DataFrame()
sub['id'] = test_data["id"]
sub["target"] = sub_preds
sub.to_csv("submission_10262017_lgb_5s_with_0.01_.csv", index=False, float_format="%.9f")

In [150]:
sub_df.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_car_05_cat_avg,ps_car_06_cat_avg,ps_car_07_cat_avg,ps_car_08_cat_avg,ps_car_09_cat_avg,ps_car_10_cat_avg,ps_car_11_cat_avg,ps_reg_01_plus_ps_car_02_cat_avg,ps_reg_01_plus_ps_car_04_cat_avg,target
0,0,1,8,1,0,0,1,0,0,0,...,0.031709,0.034185,0.034766,0.034735,0.036319,0.036464,0.030628,0.030048,0.028365,0.049053
1,4,2,5,1,0,0,0,0,1,0,...,0.040111,0.031720,0.034766,0.034735,0.033344,0.036464,0.026667,0.038214,0.038347,0.050971
2,5,1,3,0,0,0,0,0,1,0,...,0.031709,0.034294,0.034766,0.034735,0.036319,0.036464,0.036266,0.030893,0.030549,0.050628
3,0,1,6,0,0,1,0,0,0,0,...,0.031709,0.034185,0.034766,0.034735,0.036319,0.036464,0.034292,0.022851,0.020197,0.031003
4,5,1,7,0,0,0,0,0,1,0,...,0.031709,0.031720,0.034766,0.034735,0.036319,0.036464,0.038289,0.038214,0.038347,0.073665
